# 데이터분석프로그래밍 프로젝트
## 컴퓨터공학과 201911771 임영서

### 데이터: 아마존 패션 리뷰 데이터  
데이터 다운로드: http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz

### json.gz 파일을 열기 위한 작업

In [1]:
import pandas as pd

In [2]:
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient = 'index')

In [3]:
review = getDF('./AMAZON_FASHION.json.gz')

In [4]:
type(review), review.shape

(pandas.core.frame.DataFrame, (883636, 12))

In [5]:
review

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3,NaN,NaN
2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2.0,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3.0,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
883631,5.0,True,"02 21, 2017",A1ZSB2Q144UTEY,B01HJHTH5U,Amazon Customer,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,1487635200,NaN,NaN,NaN
883632,5.0,True,"11 25, 2016",A2CCDV0J5VB6F2,B01HJHTH5U,Amazon Customer,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,1480032000,2,NaN,NaN
883633,3.0,True,"11 10, 2016",A3O90PACS7B61K,B01HJHTH5U,Fabfifty,Too big in the chest area!,Three Stars,1478736000,NaN,NaN,NaN
883634,3.0,True,"11 10, 2016",A2HO94I89U3LNH,B01HJHF97K,Mgomez,"Too clear in the back, needs lining",Three Stars,1478736000,NaN,NaN,NaN


In [6]:
review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883636 entries, 0 to 883635
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         883636 non-null  float64
 1   verified        883636 non-null  bool   
 2   reviewTime      883636 non-null  object 
 3   reviewerID      883636 non-null  object 
 4   asin            883636 non-null  object 
 5   reviewerName    883544 non-null  object 
 6   reviewText      882403 non-null  object 
 7   summary         883103 non-null  object 
 8   unixReviewTime  883636 non-null  int64  
 9   vote            79900 non-null   object 
 10  style           304569 non-null  object 
 11  image           28807 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 81.7+ MB


## 데이터의 생김새
- overall: 별점(float)
- verified: 구매 확인 여부
- reviewTime: 리뷰를 적은 시간
- reviewerID: 리뷰를 적은 사람
- asin: 산 물건의 코드
- reviewerName: 리뷰를 적은 사람의 이름
- reviewText: 리뷰로 적은 내용
- summary: 요약
- unixReviewTime: 유닉스 시간으로 리뷰를 적은 시간
- vote: 받은 좋아요 수?
- style: 상품의 스타일
- image: 상품의 이미지 url

## 데이터로 유저의 rating을 예측... 하기 전에 데이터 전처리

### 1. 사용할 feature와 버릴 feature 구분

reviewTime(리뷰를 적은 시간)과 unixReviewTime(유닉스 시간으로 리뷰를 적은 시간) 확인

In [50]:
# reviewTime과 unixReviewTime 형식과 예시 데이터 확인
type(review['reviewTime'][0]), review['reviewTime'][0], type(review['unixReviewTime'][0]), review['unixReviewTime'][0]

(str, '10 20, 2014', numpy.int64, 1413763200)

In [53]:
# reviewTime은 str이기 때문에 str to datetime
# unixReviewTime은 int이기 때문에 int to datetime
from datetime import datetime
review['reviewTime'] = pd.to_datetime(review['reviewTime'], format='%m %d, %Y')
review['unixReviewTime'] = pd.to_datetime(review['unixReviewTime'], unit='s')

In [64]:
# reviewTime과 unixReviewTime이 같지 않은 데이터 확인
review[review['reviewTime'] != review['unixReviewTime']]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image


In [67]:
# 모두 다 같으므로 unixReviewTime 삭제
del review['unixReviewTime']

reviwerID(리뷰를 적은 사람)와 reviewerName(리뷰를 적은 사람의 이름) 확인

In [81]:
# 둘 다 리뷰를 적은 사람에 대한 정보를 표시한다.
# 중복이 가능한 reviewerName을 삭제하고 reviewrID를 숫자형으로 바꿔준다.

del review['reviewerName']

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
review['reviewerID'] = enc.fit_transform(review['reviewerID'])

asin(산 물건의 코드) 확인

In [85]:
review['asin'][:10]

0    7106116521
1    7106116521
2    7106116521
3    7106116521
4    7106116521
5    7106116521
6    7106116521
7    B00007GDFV
8    B00007GDFV
9    B00007GDFV
Name: asin, dtype: object

In [86]:
# 역시 asin도 숫자형으로 바꿔준다.
enc_asin = LabelEncoder()
review['asin'] = enc.fit_transform(review['asin'])

reviewText(리뷰로 적은 내용)와 summary(요약) 확인

In [89]:
# 타입 확인
type(review['reviewText'][0]), review['reviewText'][0], type(review['summary'][0]), review['summary'][0]

(str, 'Exactly what I needed.', str, 'perfect replacements!!')

In [90]:
# 텍스트 처리 불가능... 따라서 두 feature 다 삭제
del review['reviewText']
del review['summary']

vote(받은 좋아요 수), style(상품의 스타일), image(상품의 이미지 url) 확인

In [95]:
# vote
print('vote: ', type(review['vote'][0]), review['vote'][0])
# style
print('style: ', type(review['style'][0]), review['style'][0])
# image
print('image: ', type(review['image'][0]), review['image'][0])

vote:  <class 'float'> nan
style:  <class 'float'> nan
image:  <class 'float'> nan


In [96]:
# 오잉...? 셋 다 nan이 나오다니..
# nan의 비율을 계산해보자
print('vote: ', review['vote'].isna().sum() / len(review['vote']))
print('style: ', review['style'].isna().sum() / len(review['style']))
print('image: ', review['image'].isna().sum() / len(review['image']))

vote:  0.9095781520897745
style:  0.6553230063057639
image:  0.9673994721808528


In [97]:
# nan이 너무 많으므로... 세 feature 다 삭제
del review['vote']
del review['style']
del review['image']

verified(구매 확인 여부) 확인

In [100]:
# True False인 boolean으로 이루어져 있으므로
# 1 0 으로 바꿔주자
num_veri = [1 if i else 0 for i in range(len(review['verified']))]

In [102]:
review['verified'] = num_veri

In [104]:
# 최종 사용하게 될 data
review.head()

,overall,verified,reviewTime,reviewerID,asin
0,5.0,0,2014-10-20,72476,10
1,2.0,1,2014-09-28,469548,10
2,4.0,1,2014-08-25,323626,10
3,2.0,1,2014-08-24,365298,10
4,3.0,1,2014-07-27,597016,10


In [110]:
# 혹시 있을 중복을 제거한다.
review = review.drop_duplicates(keep='first')
review = review.reset_index(drop=True)

In [111]:
# 진짜 최종 data
review.head()

,overall,verified,reviewTime,reviewerID,asin
0,5.0,0,2014-10-20,72476,10
1,2.0,1,2014-09-28,469548,10
2,4.0,1,2014-08-25,323626,10
3,2.0,1,2014-08-24,365298,10
4,3.0,1,2014-07-27,597016,10


In [112]:
# data 크기
review.shape

(875699, 5)